# saliency maps




### downloads

In [ ]:
!wget -O cat1.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/cat1.jpg
!wget -O cat2.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/cat2.jpg
!wget -O catanddog.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/catanddog.jpg
!wget -O dog1.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/dog1.jpg
!wget -O dog2.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/dog2.jpg

!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1kipXTxesGJKGY1B8uSPRvxROgOH90fih' -O 0_epochs.h5
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1oiV6tjy5k7h9OHGTQaf0Ohn3FmF-uOs1' -O 15_epochs.h5


### imports

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import cv2
import keras
from keras.models import Sequential,Model
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,GlobalAveragePooling2D
from google.colab import drive, files

### download and prepare the dataset.



In [ ]:
train_data = tfds.load('cats_vs_dogs', split='train[:80%]', as_supervised=True)
validation_data = tfds.load('cats_vs_dogs', split='train[80%:90%]', as_supervised=True)
test_data = tfds.load('cats_vs_dogs', split='train[-10%:]', as_supervised=True)

#### preprocessing function.


In [ ]:
def augmentimages(image, label):
  image = tf.cast(image, tf.float32)
  image = (image / 255)
  image = tf.image.resize(image, (300, 300))
  return image, label

#### preprocess the training set.


In [ ]:
augmented_training_data = train_data.map(augmentimages)
augmented_test_data = test_data.map(augmentimages)
augmented_validation_data = validation_data.map(augmentimages)

#### create batches of the training set.

In [ ]:
BATCH_SIZE = 64
train_batches = augmented_training_data.batch(BATCH_SIZE)
test_batches = augmented_test_data.batch(BATCH_SIZE)
validation_batches = augmented_validation_data.batch(BATCH_SIZE)

### cats vs dogs classifier 

In [ ]:
model = Sequential()
model.add(Conv2D(16, input_shape=(300, 300, 3), kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32,kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3,3 ), activation='relu', padding='same'))
model.add(GlobalAveragePooling2D())
model.add(Dense(2, activation='softmax'))

model.summary()

### function to generate the saliency map.

In [ ]:
def do_salience(image, model, label, prefix):
  img = cv2.imread(image)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (300, 300)) / 255.0
  tensor_image = np.expand_dims(img, axis=0)

  num_classes = 2
  expected_output = tf.one_hot([label] * tensor_image.shape[0], num_classes)

  with tf.GradientTape() as tape:
    inputs = tf.cast(tensor_image, tf.float32)
    tape.watch(inputs)
    predictions = model(inputs)
    loss = tf.keras.losses.categorical_crossentropy(
        expected_output, predictions
    )

  gradients = tape.gradient(loss, inputs)
  grayscale_tensor = tf.reduce_sum(tf.abs(gradients), axis=-1)
  normalized_tensor = tf.cast(
    255
    * (grayscale_tensor - tf.reduce_min(grayscale_tensor))
    / (tf.reduce_max(grayscale_tensor) - tf.reduce_min(grayscale_tensor)),
    tf.uint8,
)

  normalized_tensor = tf.squeeze(normalized_tensor)

  plt.figure(figsize=(8, 8))
  plt.axis('off')
  plt.imshow(normalized_tensor, cmap='gray')
  plt.show()

  gradient_color = cv2.applyColorMap(normalized_tensor.numpy(), cv2.COLORMAP_HOT)
  gradient_color = gradient_color / 255.0
  super_imposed = cv2.addWeighted(img, 0.5, gradient_color, 0.5, 0.0)

  plt.figure(figsize=(8, 8))
  plt.imshow(super_imposed)
  plt.axis('off')
  plt.show()

  salient_image_name = prefix + image
  normalized_tensor = tf.expand_dims(normalized_tensor, -1)
  normalized_tensor = tf.io.encode_jpeg(normalized_tensor, quality=100, format='grayscale')
  writer = tf.io.write_file(salient_image_name, normalized_tensor)

## generate saliency maps with untrained model.

In [ ]:
model.load_weights('0_epochs.h5')

In [ ]:
do_salience("catanddog.jpg", model, 0, "epoch0_salient")

### configure the model for training.

In [ ]:
rms_prop = tf.keras.optimizers.RMSprop(
    learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,
    name='RMSprop')

In [ ]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

### train model.

#### mount google drive.

In [ ]:
drive.mount('/content/drive/', force_remount=True)

In [ ]:
checkpoint_filepath = '/content/drive/My Drive/coursera/saliency_maps/model.ckpt'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, 
                                                               monitor='accuracy', mode='max', save_best_only=True)

#### load model weights.

In [ ]:
model.load_weights('15_epochs.h5')

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
EPOCHS = 100

steps_per_epoch = len(train_batches) // BATCH_SIZE
validation_steps = len(validation_batches) // BATCH_SIZE
test_steps = len(test_batches) // BATCH_SIZE

history = model.fit(train_batches,
                    steps_per_epoch=steps_per_epoch, validation_data=validation_batches, 
                    validation_steps=validation_steps, epochs=EPOCHS,
                    callbacks=[model_checkpoint_callback])

### generate saliency maps.

In [ ]:
do_salience("cat1.jpg", model, 1, "salient")
do_salience("cat2.jpg", model, 2, "salient")
do_salience("catanddog.jpg", model, 0, "salient")
do_salience("dog1.jpg", model, 3, "salient")
do_salience("dog2.jpg", model, 4, "salient")

### zip the images.

In [ ]:
from zipfile import ZipFile

!rm images.zip

filenames = ['cat1.jpg', 'cat2.jpg', 'catanddog.jpg', 'dog1.jpg', 'dog2.jpg']

with ZipFile('images.zip','w') as zip:
  for file in filenames:
    zip.write('salient' + file)

print("images.zip generated!")

### saliency maps on 95 epochs.

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=14vFpBJsL_TNQeugX8vUTv8dYZxn__fQY' -O 95_epochs.h5

model.load_weights('95_epochs.h5')

do_salience('cat1.jpg', model, 0, "epoch95_salient")
do_salience('cat2.jpg', model, 0, "epoch95_salient")
do_salience('catanddog.jpg', model, 0, "epoch95_salient")
do_salience('dog1.jpg', model, 1, "epoch95_salient")
do_salience('dog2.jpg', model, 1, "epoch95_salient")

## References
##### Coursera - Advanced computer vision with tensorflow [course](https://www.coursera.org/learn/advanced-computer-vision-with-tensorflow/).